In [1]:
import torch

In [2]:
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]

t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [3]:
t_u.shape

torch.Size([11])

In [4]:
import torch.optim as optim

dir(optim)

['ASGD',
 'Adadelta',
 'Adafactor',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'LBFGS',
 'NAdam',
 'Optimizer',
 'RAdam',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_adafactor',
 '_functional',
 'lr_scheduler',
 'swa_utils']

In [5]:
import torch.nn as nn

In [6]:
t_c.shape

torch.Size([11])

In [7]:
t_u_unsq = t_u.unsqueeze(1)  # shape: (11, 1)
weights = torch.ones([1, 1], requires_grad=True)

model = nn.Linear(in_features=1, out_features=1)

with torch.no_grad():
    model.weight.data.fill_(1.0)

lr = 1e-3  # 0.01
optimizer = optim.RMSprop(model.parameters(), lr=lr)
criterion = nn.HuberLoss()

epochs = 5000

for epoch in range(epochs):
    output = model(t_u_unsq)
    loss = criterion(t_c.unsqueeze(1), output)
    if (epoch + 1) % 1000 == 0:
        print(loss)
    loss.backward()
    optimizer.step()

model.weight, model.bias

tensor(11.3626, grad_fn=<HuberLossBackward0>)
tensor(7.6580, grad_fn=<HuberLossBackward0>)
tensor(3.5064, grad_fn=<HuberLossBackward0>)
tensor(4.0536, grad_fn=<HuberLossBackward0>)
tensor(3.1213, grad_fn=<HuberLossBackward0>)


(Parameter containing:
 tensor([[0.3424]], requires_grad=True),
 Parameter containing:
 tensor([-5.1923], requires_grad=True))

In [8]:
t_u.shape

torch.Size([11])

In [9]:
from torch.utils.data import TensorDataset

data = TensorDataset(t_u, t_c)

In [10]:
from torch.utils.data import random_split

train, val = random_split(data, [0.8, 0.2], generator=torch.Generator().manual_seed(42))

In [11]:
data.tensors[0][train.indices]

tensor([33.9000, 48.4000, 60.4000, 58.2000, 55.9000, 68.4000, 35.7000, 21.8000,
        81.9000])

In [12]:
X_train = data.tensors[0][train.indices].unsqueeze(1)
y_train = data.tensors[1][train.indices].unsqueeze(1)

X_val = data.tensors[0][val.indices].unsqueeze(1)
y_val = data.tensors[1][val.indices].unsqueeze(1)

In [13]:
X_train.shape

torch.Size([9, 1])

In [19]:
t_u_unsq = t_u.unsqueeze(1)  # shape: (11, 1)
bias = torch.zeros(1, requires_grad=True)
weights1 = torch.ones(1, 1, requires_grad=True)
weights2 = torch.ones(1, 1, requires_grad=True)
model = lambda x: weights2 * x + weights1 * x + bias


lr = 1e-2  # 0.01
optimizer = optim.Adam([weights1, weights2, bias], lr=lr)
criterion = nn.HuberLoss()

epochs = 3000

for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    if (epoch + 1) % 10000 == 0:
        print("train loss:", loss)
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        output = model(X_val)
        loss = criterion(output, y_val)
        if (epoch + 1) % 300 == 0:
            print("val loss:", loss, "\n")

val loss: tensor(2.8369) 

val loss: tensor(2.7152) 

val loss: tensor(2.5628) 

val loss: tensor(2.3889) 

val loss: tensor(2.1995) 

val loss: tensor(2.1499) 

val loss: tensor(2.1659) 

val loss: tensor(2.0078) 

val loss: tensor(1.9523) 

val loss: tensor(1.9278) 

